In [1]:
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
import json
import torch

In [2]:
import dataset
import algorithm

In [15]:
CONFIG = json.load(open('config.json'))

In [4]:
data_eval = dataset.download_dataset(30)

Into folder: Dataset/Not/Argetina
	Segmentize file: CBERS_4A_WPM_20210416_219_155_L2_BAND1.tif
		File Shape: (14879, 14498, 3)
		Images div: 1/537932, 0%
		Images div: 53794/537932, 10%
		Images div: 107587/537932, 20%
		Images div: 161380/537932, 30%
	Segmentize file: CBERS_4A_WPM_20210416_219_155_L2_BAND2.tif
		File Shape: (14879, 14498, 3)
		Images div: 1/537932, 0%
		Images div: 53794/537932, 10%
		Images div: 107587/537932, 20%
		Images div: 161380/537932, 30%
	Segmentize file: CBERS_4A_WPM_20210416_219_155_L2_BAND3.tif
		File Shape: (14879, 14498, 3)
		Images div: 1/537932, 0%
		Images div: 53794/537932, 10%
		Images div: 107587/537932, 20%
		Images div: 161380/537932, 30%
	Segmentize file: CBERS_4A_WPM_20210416_219_155_L2_BAND4.tif
		File Shape: (14879, 14498, 3)
		Images div: 1/537932, 0%
		Images div: 53794/537932, 10%
		Images div: 107587/537932, 20%
		Images div: 161380/537932, 30%
	JoinBands
Into folder: Dataset/Yes/Argentina
	Segmentize file: CBERS_4A_WPM_20210119_224_147_

In [6]:
normalized_data = dataset.Normalizer.transform(data_eval)

In [7]:
data_x, data_y = dataset.split_data_x_y(normalized_data)

In [8]:
dataset_eval = dataset.TimeSeriesDataset(data_x, data_y)

In [10]:
dataloader_eval = DataLoader(dataset_eval, CONFIG["training"]["batch_size"], shuffle=True)

In [16]:
convnet = algorithm.CNNModel(config_model=CONFIG["model"], config_training=CONFIG["training"])

In [18]:
convnet.load_state_dict(torch.load(CONFIG["model"]["path"], map_location="cpu"))

<All keys matched successfully>

In [20]:
correct = 0
total = 0

with torch.no_grad():
    for idx, (images, labels) in enumerate(dataloader_eval):
        images = images.to(CONFIG["training"]["device"])
        labels = labels.to(CONFIG["training"]["device"])
        outputs = convnet(images)
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the {0} test images: {1} %'.format(total, 100*correct/total))

c:\python38\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Accuracy of the network on the 316439 test images: 76.7114672970146 %
